In [1]:
import sys
import os
import numpy as np
import multiprocessing
import dill
import matplotlib.pyplot as plt
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_params

from multi_group_simulation import MultiGroupSimulation

In [2]:
def get_cum_hosp(df):
    return df[['severity_3', 'severity_2']].iloc[df.shape[0] - 1].sum()

def get_cum_outside_infections(df):
    return df['cumulative_outside_infections'].iloc[df.shape[0] - 1].sum()

def get_cum_infections(df):
    return df[['cumulative_mild', 'cumulative_severe']].iloc[df.shape[0] - 1].sum()

def get_cum_inf_trajectory(df):
    return np.sum(df[['cumulative_mild', 'cumulative_severe']], axis=1)

def get_cum_hosp_trajectory(df):
    return np.sum(df[['severity_3', 'severity_2']], axis=1)

def total_infections(list_sim_dfs):
    total = 0
    for sim_df in list_sim_dfs:
        total += get_cum_infections(sim_df)
    return total

def total_hosp(list_sim_dfs):
    total = 0
    for sim_df in list_sim_dfs:
        total += get_cum_hosp(sim_df)
    return total


In [3]:
def get_trajectories(params_list, interaction_matrix, group_names, test_policy, T, n):
    assert len(params_list) == len(test_policy)
    
    for index, params in enumerate(params_list):
        params['expected_contacts_per_day'] = interaction_matrix[index, index]
        params['test_population_fraction'] = test_policy[index]

    sim = MultiGroupSimulation(params_list, interaction_matrix, group_names)

    inf_trajectories = list()
    hosp_trajectories = list()
    
    for _ in range(n):
        sim.run_new_trajectory(T)
        for group in sim.sims:
            df = group.sim_df
            inf_trajectories.append(get_cum_inf_trajectory(df))
            hosp_trajectories.append(get_cum_hosp_trajectory(df))

    return inf_trajectories, hosp_trajectories

In [4]:
# Fall Semester Group Config

# Loading group params
base_directory = '../src/simulations_v2/params/jan4calibration/'

ug_ga_params = load_params(base_directory + 'group_1_students_post_movein_private.yaml')[1]
ug_other_params = load_params(base_directory + 'group_2_students_post_movein_private.yaml')[1]
gs_params = load_params(base_directory + 'group_3_students_post_movein_private.yaml')[1]
fs_params = load_params(base_directory + 'faculty_staff_post_movein_private.yaml')[1]
params_list = [ug_ga_params.copy(), ug_other_params.copy(), gs_params.copy(), fs_params.copy()]

/home/aaj54/group-testing/src/simulations_v2/load_params.py:136: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  params = yaml.load(f)
/home/aaj54/group-testing/src/simulations_v2/load_params.py:45: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  age_sev_params = yaml.load(f)


In [5]:
# Scaling of the interaction matrix is wrong, made up number of f/s -> f/s contact
interaction_matrix = np.array([[161/125, 2/44, 0,0],
                              [8/125, 7/44, 0,0],
                              [0,0,1/15,0],
                              [0,0,0,1]])

# Transmission prob: 0.3

# Making up initial ID prevalence
id_prevalence = 0.01
for params in params_list:
    params['initial_ID_prevalence'] = id_prevalence

# adding population size
params_list[0]['population_size'] = 3533
params_list[1]['population_size'] = 8434
params_list[2]['population_size'] = 6202
params_list[3]['population_size'] = 10000

group_names = ['UG (Greek, Athlete)', 'UG (other)', 'GS', 'Faculty/Staff']

In [6]:
np.random.seed(2022)
test_policy = [2/7,2/7,1/7,1/7]

inf_trajs, hosp_trajs = get_trajectories(params_list, interaction_matrix, group_names, test_policy, 112, 5)

In [21]:
params_list[0]

{'severity_prevalence': array([0.47808584, 0.50907724, 0.00940271, 0.00343422]),
 'max_time_exposed': 7,
 'exposed_time_function': <function analysis_helpers.poisson_waiting_function.<locals>.<lambda>(n)>,
 'max_time_ID': 8,
 'ID_time_function': <function analysis_helpers.poisson_waiting_function.<locals>.<lambda>(n)>,
 'max_time_SyID_mild': 20,
 'SyID_mild_time_function': <function analysis_helpers.poisson_waiting_function.<locals>.<lambda>(n)>,
 'max_time_SyID_severe': 20,
 'SyID_severe_time_function': <function analysis_helpers.poisson_waiting_function.<locals>.<lambda>(n)>,
 'mild_symptoms_daily_self_report_p': 0,
 'cases_isolated_per_contact': 1.329,
 'cases_quarantined_per_contact': 3.304,
 'contact_trace_testing_frac': 1,
 'contact_tracing_delay': 1,
 'sample_QI_exit_function': <function analysis_helpers.binomial_exit_function.<locals>.<lambda>(n)>,
 'sample_QS_exit_function': <function analysis_helpers.binomial_exit_function.<locals>.<lambda>(n)>,
 'daily_outside_infection_p': 